# Note

In [1]:
# model on fullnames but make sure that last names aligned at the same position
# this is to overcome the fullname model only focus on firstname part

# Preprocess the data

In [2]:
# change keras default GPU
# os.environ["CUDA_VISIBLE_DEVICES"]="0" # first gpu
# os.environ["CUDA_VISIBLE_DEVICES"]="1" # second gpu
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # runs in cpu

In [3]:
# pip install keras-tuner

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd drive/MyDrive/rethnicity/

/content/drive/MyDrive/rethnicity


In [6]:
import tensorflow.keras as keras
import tensorflow as tf
print(keras.__version__)
#print(tf.__version__)
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import string

2.6.0


In [7]:
df = pd.read_csv('name_undersampled.csv')

In [8]:
df

,name_combine,name_first,name_last,race,gender
0,Sarah Rahim,Sarah,Rahim,2,F
1,Amy Mai,Amy,Mai,2,F
2,Sarah Tsang,Sarah,Tsang,2,F
3,Neelima Gopal,Neelima,Gopal,2,F
4,Maria Luna,Maria,Luna,2,F
...,...,...,...,...,...
837051,Michael Ambrose,Michael,Ambrose,5,M
837052,Duane Campbell,Duane,Campbell,5,M
837053,Stephen Rosenfeld,Stephen,Rosenfeld,5,M
837054,Gregory Lynch,Gregory,Lynch,5,M


In [9]:
# create ASCII dictionary
chars = ['E'] + [chr(i) for i in range(97,123)] + [' ', 'U']
id2char = {i:j for i,j in enumerate(chars)}
char2id = {j:i for i,j in enumerate(chars)}

In [10]:
# the characters here are all ASCII, good
# for name in df['name_combine'].tolist():
#     namechars = list(name)
#     for nc in namechars:
#         if nc not in char2id:
#             print(nc)

In [11]:
def name2id(name, l = 10):
    ids = [0] * l
    for i, c in enumerate(name):
        if i < l:
            if c.isalpha():
                ids[i] = char2id.get(c, char2id['U'])
            elif c in string.punctuation:
                ids[i] = char2id.get(c, char2id[' '])
            else:
                ids[i] = char2id.get(c, char2id['U'])
    return ids

In [12]:
X = [name2id(fn.lower()) + name2id(ln.lower()) for fn, ln in zip(df['name_first'], df['name_last'])]
y = [int(i) for i in df['race'].tolist()]

In [13]:
# convert the output (y) from 2-5 to 0-3
y = [i-2 for i in y]

In [14]:
# Split train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [15]:
# check the distribution
print('distribution of all data points', {i: y.count(i) for i in set(y)})
print('distribution of all training data', {i: y_train.count(i) for i in set(y)})
print('distribution of all testing data', {i: y_test.count(i) for i in set(y)})

distribution of all data points {0: 209264, 1: 209264, 2: 209264, 3: 209264}
distribution of all training data {0: 167403, 1: 167360, 2: 167324, 3: 167557}
distribution of all testing data {0: 41861, 1: 41904, 2: 41940, 3: 41707}


# Train the model

In [16]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM, Bidirectional

num_words = len(id2char)
feature_len = 20 # cut texts after this number of words (among top max_features most common words)
batch_size = 512

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = 4 # np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

669644 train sequences
167412 test sequences
Pad sequences (samples x time)
X_train shape: (669644, 20)
X_test shape: (167412, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (669644, 4)
y_test shape: (167412, 4)


In [17]:
# simple train-test
# first build
model = Sequential()
model.add(Embedding(num_words, 256, input_length=feature_len))
# try out bi-directional LSTM
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, dropout=0.2)))
model.add(Dense(num_classes, activation='softmax'))

# choose between learning rates
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# train model
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_split=0.2, verbose=1, callbacks=[callback])
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

Epoch 1/10
1047/1047 [==============================] - 484s 451ms/step - loss: 0.9208 - accuracy: 0.6113 - val_loss: 0.7575 - val_accuracy: 0.7003
Epoch 2/10
1047/1047 [==============================] - 475s 453ms/step - loss: 0.7008 - accuracy: 0.7282 - val_loss: 0.6569 - val_accuracy: 0.7494
Epoch 3/10
1047/1047 [==============================] - 475s 454ms/step - loss: 0.6244 - accuracy: 0.7641 - val_loss: 0.6216 - val_accuracy: 0.7667
Epoch 4/10
1047/1047 [==============================] - 475s 454ms/step - loss: 0.5793 - accuracy: 0.7834 - val_loss: 0.6028 - val_accuracy: 0.7756
Epoch 5/10
1047/1047 [==============================] - 474s 453ms/step - loss: 0.5440 - accuracy: 0.7983 - val_loss: 0.5955 - val_accuracy: 0.7813
Epoch 6/10
1047/1047 [==============================] - 476s 454ms/step - loss: 0.5120 - accuracy: 0.8105 - val_loss: 0.5986 - val_accuracy: 0.7835
Epoch 7/10
1047/1047 [==============================] - 476s 454ms/step - loss: 0.4831 - accuracy: 0.8214 - val_

In [18]:
# now lets' test
y_pred = model.predict(X_test, batch_size=batch_size, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(np.argmax(y_test, axis=1), y_pred_bool))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred_bool))

327/327 [==============================] - 50s 148ms/step
              precision    recall  f1-score   support

           0       0.86      0.76      0.81     41861
           1       0.76      0.73      0.75     41904
           2       0.85      0.87      0.86     41940
           3       0.66      0.74      0.70     41707

    accuracy                           0.78    167412
   macro avg       0.78      0.78      0.78    167412
weighted avg       0.78      0.78      0.78    167412

[[31824  2338  2980  4719]
 [ 1461 30797  1110  8536]
 [ 1706  1062 36637  2535]
 [ 1943  6496  2386 30882]]


In [19]:
model.save('fullname_aligned.h5', include_optimizer=False)
model.save('fullname_aligned_opt.h5')

# Distill the Model

In [20]:
# TODO: shall we try model distillation for compressing the model size?
# so that we will have smaller model to work with
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [21]:
# Create the student
student = keras.Sequential(
    [
        Embedding(num_words, 32, input_length=feature_len),
        Bidirectional(LSTM(16, return_sequences=True, dropout=0.2)),
        Bidirectional(LSTM(16, return_sequences=True, dropout=0.2)),
        Bidirectional(LSTM(16, return_sequences=True, dropout=0.2)),
        Bidirectional(LSTM(16, dropout=0.2)),
        Dense(num_classes, activation='softmax')
    ],
    name="student",
)

In [22]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(X_train, y_train, epochs=40)

# Evaluate student on test dataset
distiller.evaluate(X_test, y_test)

Epoch 1/40
20927/20927 [==============================] - 693s 32ms/step - categorical_accuracy: 0.6034 - student_loss: 0.9379 - distillation_loss: 3.7233e-04
Epoch 2/40
20927/20927 [==============================] - 707s 34ms/step - categorical_accuracy: 0.6674 - student_loss: 0.8243 - distillation_loss: 2.9024e-04
Epoch 3/40
20927/20927 [==============================] - 708s 34ms/step - categorical_accuracy: 0.6830 - student_loss: 0.7935 - distillation_loss: 2.6745e-04
Epoch 4/40
20927/20927 [==============================] - 715s 34ms/step - categorical_accuracy: 0.6910 - student_loss: 0.7768 - distillation_loss: 2.5557e-04
Epoch 5/40
20927/20927 [==============================] - 713s 34ms/step - categorical_accuracy: 0.6962 - student_loss: 0.7667 - distillation_loss: 2.4801e-04
Epoch 6/40
20927/20927 [==============================] - 710s 34ms/step - categorical_accuracy: 0.6998 - student_loss: 0.7603 - distillation_loss: 2.4307e-04
Epoch 7/40
20927/20927 [======================

[0.7372231483459473, 0.5687428116798401]

In [ ]:
y_pred = distiller.student.predict(X_test, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(np.argmax(y_test, axis=1), y_pred_bool))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred_bool))

 197/5232 [>.............................] - ETA: 43s

In [ ]:
distiller.student.save('fullname_aligned_distill_opt.h5')
distiller.student.save('fullname_aligned_distill.h5', include_optimizer=False)

# Tune the Model

In [ ]:
model.summary()

In [ ]:
model.get_config()

In [ ]:
model.optimizer

In [ ]:
from keras import backend as K
K.eval(model.optimizer.lr)

In [ ]:
# from google.colab import files
# files.download('fullname_2_ethnicity_bilstm.h5') 